# Digital
---

1. Data Preparation
2. Model Comparison
- K-mean
- LDA
- NMF
- Top2Vec > still in progress and will continue if time permits
- BERTopic > give up BERTopic because of time constraints

## 0. Import Libraries

In [2]:
# Import Basic Libraries
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Import Sklearn Libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import NMF
from sklearn.cluster import KMeans


# Import NLP Libraries
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary
from pythainlp.tokenize import sent_tokenize, word_tokenize
from pythainlp.corpus import thai_stopwords
import re
from tqdm import tqdm 
import pyLDAvis
import pyLDAvis.gensim_models
from top2vec import Top2Vec


# Set default Thai font
mpl.font_manager.fontManager.addfont('./THSarabunNew/THSarabunNew.ttf')
mpl.rc('font', family='TH Sarabun New', size=20)

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


## 1. Data Preparation

### 1.1 Topic Selection

In [61]:
digitalb = pd.read_json('../datasets/bkkbiz_digital_processed.json')
digitalm = pd.read_json('../datasets/matichon_digital_processed.json')

In [63]:
digitalb.head()

,ID,date,category,title,description,article,url,title_count,title_tokenize,description_count,description_tokenize,article_count,article_tokenize
0,620015,2014-11-27 10:08:00,การเงิน-การลงทุน,เนชั่นจ่อปรับค่าโฆษณา20%,"""เนชั่น มัลติมีเดีย กรุ๊ป""หวังรายได้ปีหน้าแตะ3...",นางสาวดวงกมล โชตะนา กรรมการผู้อำนวยการ บริษัท ...,https://www.bangkokbiznews.com/news/detail/620015,6,"[เนชั่น, จ่อ, ค่าโฆษณา]",25,"[เนชั่น, มัลติมีเดีย, กรุ๊ป, หวัง, รายได้, ปีห...",738,"[ดวงกมล, โช, ตะ, นา, กรรมการ, ผู้อำนวยการ, บริ..."
1,620023,2014-11-27 10:26:00,การเมือง,ทุ่ม80ล้านเปิดเวที77จว.,สปช.ลุยฟังความเห็นปฏิรูป-คิกออฟ1ธ.ค. รวบ2ผู้ต้...,สปช.ทุ่มงบ 80 ล้าน เปิด 1.3 พันเวทีปฏิรูป รับฟ...,https://www.bangkokbiznews.com/news/detail/620023,7,"[ทุ่ม, ล้าน, เวที, จว.]",22,"[สปช., ลุย, ฟัง, ความเห็น, ปฏิรูป, คิก, ออฟ, ธ...",1381,"[สปช., ทุ่ม, งบ, ล้าน, พัน, เวที, ปฏิรูป, รับฟ..."
2,620025,2014-11-27 10:35:00,ธุรกิจ,NOW26ปักหลักสยามสแควร์ เปิดสตูดิโอใหม่พันตรม.-...,เปิดแผนปี 2558 ดิจิทัลทีวี NOW26 เตรียมเปิดสตู...,เพิ่มช่องทางเข้าถึงคนรุ่นใหม่ สร้างประสบการณ์ร...,https://www.bangkokbiznews.com/news/detail/620025,13,"[NOW, ปักหลัก, สยามสแควร์, สตูดิโอ, พัน, ตรม, ...",21,"[แผน, ปี, ดิจิทัล, ทีวี, NOW, เตรียม, สตูดิโอ,...",631,"[ช่องทาง, เข้าถึง, คนรุ่นใหม่, สร้าง, ประสบการ..."
3,620054,2014-11-27 11:08:00,การเงิน-การลงทุน,NBC - ซื้อ,เตรียมปรับเพิ่มค่าโฆษณาช่อง Nation TV ขึ้น 30-50%,ปรับเพิ่มมูลค่าเหมาะสมเป็น 6 บาท เราปรับเพิ่มม...,https://www.bangkokbiznews.com/news/detail/620054,3,"[NBC, ซื้อ]",12,"[เตรียม, ค่าโฆษณา, ช่อง, Nation, TV]",657,"[มูลค่า, เหมาะสม, บาท, มูลค่า, เหมาะสม, หุ้น, ..."
4,620065,2014-11-27 11:37:00,ธุรกิจ,'ซิคเว่'เสนอแผน'ดิจิทัลอีโคโนมี',"""ซิคเว่"" เล็งผนึก ""เอไอเอส ทรู"" พร้อมควง ""ทีโอ...","หวังสานฝันนโยบายรัฐสัมฤทธิผล รอนัดวันเข้าพบ ""ห...",https://www.bangkokbiznews.com/news/detail/620065,12,"[ซิ, คเว่', เสนอ, แผน, ดิจิทัล, อี, โค, โน]",28,"[ซิ, คเว่"", เล็ง, ผนึก, เอไอเอส, ทรู, ควง, ทีโ...",1329,"[หวัง, สาน, ฝัน, นโยบาย, รัฐ, สัมฤทธิผล, รอ, น..."


In [64]:
digitalm.head()

,title,article,date,category,url,id,title_count,title_tokenize,article_count,article_tokenize
0,"แห่ชมฝนดาวตก ""เจมินิดส์""คึกคัก ""ดอยอินทนนท์"" ช...",วันที่ 15 ธันวาคม นายศรันย์ โปษยะจินดา รองผู้อ...,2015-12-15 02:36:22,region,https://www.matichon.co.th/region/news_622,622,20,"[แห่, ชม, ฝนดาวตก, เจ, ส์"", คึกคัก, ดอย, อินทน...",721,"[วันที่, ธันวาคม, นา, ยศ, รัน, ย์, โป, ษ, ยะ, ..."
1,สมาชิกรุมจวกกม.คลื่นความถี่! 'วิทยา' ซัดสปท.ไม...,ถก ร่างแก้ไข พ.ร.บ.องค์กรจัดสรรคลื่นความถี่ ทำ...,2016-01-18 23:48:28,politics,https://www.matichon.co.th/politics/news_5384,5384,19,"[สมาชิก, รุม, จวก, กม., คลื่นความถี่, วิทยา, ซ...",558,"[ถก, ร่าง, พ.ร.บ., องค์กร, จัดสรร, คลื่นความถี..."
2,โครงการ 3 ประสานความร่วมมือ กสทช. กสท และ CABSAT,โครงการ 3 ประสานความร่วมมือ กสทช. กสท และ CABS...,2016-01-20 09:08:59,publicize,https://www.matichon.co.th/publicize/news_6608,6608,8,"[โครงการ, ประสาน, ความร่วมมือ, กสทช., กสท, CAB...",425,"[โครงการ, ประสาน, ความร่วมมือ, กสทช., กสท, CAB..."
3,"เอไอเอส จัดเต็ม มหกรรม ""AIS 4G ADVANCED NEW WO...","เอไอเอสขอเชิญลูกค้าและประชาชน ร่วมงานมหกรรม ""A...",2016-01-24 09:08:54,publicize,https://www.matichon.co.th/publicize/news_11306,11306,14,"[เอไอเอส, เต็ม, มหกรรม, AIS, G, ADVANCED, NEW,...",475,"[เอไอเอส, เชิญ, ลูกค้า, ประชาชน, ร่วมงาน, มหกร..."
4,"CAT เปิดศูนย์นวัตกรรม ""IOT City Innovation Ce...",บมจ. กสท โทรคมนาคม (CAT) วางโครงข่ายระบบไอทีค...,2016-01-24 12:46:00,publicize,https://www.matichon.co.th/publicize/news_11463,11463,10,"[CAT, ศูนย์, นวัตกรรม, IOT, City, Innovation, ...",404,"[บมจ., กสท, โทรคมนาคม, CAT, วาง, โครงข่าย, ระบ..."


### 1.2 Bag-of-Words

In [65]:
# Finction to store n_word in dict
def featurize(token_list):
    token_list=token_list
    features = {}
    for token in token_list:
        features[token]=1
    return features

In [66]:
digitalb_bow = digitalb['article_tokenize'].apply(featurize)
digitalm_bow = digitalm['article_tokenize'].apply(featurize)

In [67]:
digitalb_bow.shape, digitalm_bow.shape

((16464,), (10433,))

In [68]:
vectorizerb = DictVectorizer(sparse=True)
digitalb_vec = vectorizerb.fit_transform(digitalb_bow)

vectorizerm = DictVectorizer(sparse=True)
digitalm_vec = vectorizerm.fit_transform(digitalm_bow)

In [69]:
digitalb_vec, digitalm_vec

(<16464x63496 sparse matrix of type '<class 'numpy.float64'>'
 	with 2760975 stored elements in Compressed Sparse Row format>,
 <10433x59082 sparse matrix of type '<class 'numpy.float64'>'
 	with 2135687 stored elements in Compressed Sparse Row format>)

## 2. Model Comparison

### 2.1 K-Means

In [14]:
def kmeans_topic(data_vec=None, n_clusters=10):
    kmeans = KMeans(n_clusters=n_clusters, 
                    init='k-means++', 
                    max_iter=100, 
                    n_init=1, 
                    random_state=42)

    kmeans.fit(data_vec)

    order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
    
    if data_vec == digitalb_vec:
        terms = vectorizerb.get_feature_names_out()
    elif data_vec == digitalm_vec:
        terms = vectorizerm.get_feature_names_out()
    else:
        print('error')

    for i in range(n_clusters):
        print(f'Topic {i+1}')
        print('-'*10)
        for i in order_centroids[i,:10]:
            print(terms[i])
        print('\n')

In [ ]:
kmeans_topic(data_vec=digitalb_vec, n_clusters=10)

### 2.2 LDA

In [29]:
# Function for modeling with LDA
def lda_model(data=None, num_topics=None): #, topicid=None
    
    dictionary = Dictionary(data)
    corpus = [dictionary.doc2bow(txt) for txt in data]
    
    model = LdaModel(corpus=corpus, num_topics=num_topics)
    #topic = pd.DataFrame(model.get_topic_terms(topicid=1, topn=20)).rename(columns={0:'index', 1:'probability'})
    for i in range(num_topics):
        top_n = [dictionary[index] for index, prob in model.get_topic_terms(topicid=i, topn=30)]
        print(f'Topic {i+1}')
        print(top_n)
        print('-'*60)
    return model

In [36]:
# Function for modeling with LDA
def lda_model(data=None, num_topics=None): #, topicid=None
    
    dictionary = Dictionary(data)
    corpus = [dictionary.doc2bow(txt) for txt in data]
    
    model = LdaModel(corpus=corpus, num_topics=num_topics)
    perplexity = model.log_perplexity(corpus)
    #topic = pd.DataFrame(model.get_topic_terms(topicid=1, topn=20)).rename(columns={0:'index', 1:'probability'})
    print(f'LDA Model with {num_topics} topics')
    print(f'Perplexity: {perplexity}')
    print('-'*60)
    for i in range(num_topics):
        top_n = [dictionary[index] for index, prob in model.get_topic_terms(topicid=i, topn=30)]
        print(f'Topic {i+1}')
        print(top_n)
        print('-'*60)
    return model

In [30]:
# Visualization
def lda_vis(data=None, num_topics=20):
    dictionary = Dictionary(data)
    corpus = [dictionary.doc2bow(txt) for txt in data]
    model = LdaModel(corpus=corpus, num_topics=num_topics)
    pyLDAvis.enable_notebook()
    viz = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
    return viz

In [39]:
digital_5 = lda_model(data=digitalb['article_tokenize'], num_topics=5)

LDA Model with 5 topics
Perplexity: -8.081812227346278
------------------------------------------------------------
Topic 1
['ล้าน', 'บาท', 'ปี', 'บริษัท', 'ดิจิทัล', 'ธุรกิจ', 'ไทย', 'สินทรัพย์', 'เติบโต', 'ตลาด', 'ราคา', 'รายได้', 'หุ้น', 'คาด', 'เดือน', 'เพิ่มขึ้น', 'จำกัด', 'ลูกค้า', 'มูลค่า', 'ธนาคาร', 'ซื้อ', 'ลดลง', 'เงิน', 'การลงทุน', 'ขาย', 'จำนวน', 'บริการ', 'ลงทุน', 'โครงการ', 'สลาก']
------------------------------------------------------------
Topic 2
['เงิน', 'ดิจิทัล', 'วันที่', 'บิท', 'ริ', 'สลาก', 'สกุล', 'ค', 'ป', 'คอย', 'น์', 'รางวัล', 'ปี', 'คน', 'บาท', 'Bitkub', 'ระบบ', 'จำนวน', 'โท', 'จ.', 'เป๋า', 'ตรวจ', 'เมือง', 'ตัง', 'โอน', 'สลากกินแบ่งรัฐบาล', 'ออนไลน์', 'ข้อมูล', 'น.', 'พื้นที่']
------------------------------------------------------------
Topic 3
['สินค้า', 'ออนไลน์', 'คน', 'แบรนด์', 'ทำ', 'ธุรกิจ', 'ไทย', 'ลูกค้า', 'สร้าง', 'ปี', 'ตลาด', 'ผู้บริโภค', 'ดิจิทัล', 'บริษัท', 'โลก', '์', 'ดี', 'โควิด', 'แพลตฟอร์ม', 'ซื้อ', 'สำหรับ', 'ร้าน', 'อร', 'มากขึ้น', 'การ

In [37]:
digitalb_10 = lda_model(data=digitalb['article_tokenize'], num_topics=10)

LDA Model with 10 topics
Perplexity: -8.220969046722981
------------------------------------------------------------
Topic 1
['บาท', 'ล้าน', 'ปี', 'บริษัท', 'ธุรกิจ', 'หุ้น', 'ราคา', 'รายได้', 'คาด', 'เติบโต', 'เพิ่มขึ้น', 'ดิจิทัล', 'มูลค่า', 'เดือน', 'ลดลง', 'ไทย', 'ตลาด', 'จำกัด', 'ซื้อ', 'กำไร', 'ไตรมาส', 'ขาย', 'จำนวน', 'มหาชน', 'ลงทุน', 'ทีวี', 'การลงทุน', 'ต่อเนื่อง', 'ช่อง', 'โครงการ']
------------------------------------------------------------
Topic 2
['คน', 'ทำ', 'ออนไลน์', 'เรื่อง', 'โลก', 'ปี', 'สร้าง', 'ดิจิทัล', 'ธุรกิจ', 'สื่อ', 'ดี', 'เด็ก', 'งาน', 'ไทย', 'ข้อมูล', 'ดู', 'ผม', 'ข่าว', 'ยุค', 'ทำงาน', 'โควิด', 'ตัวเอง', 'ที่จะ', 'รู้', 'สินค้า', 'บริษัท', 'มากขึ้น', 'บ้าน', 'สังคม', 'เข้ามา']
------------------------------------------------------------
Topic 3
['ข้อมูล', 'วันที่', 'พื้นที่', 'ตรวจสอบ', 'ตรวจ', 'ผู้ป่วย', 'ระบบ', 'โรงพยาบาล', 'ประชาชน', 'จังหวัด', 'จ.', 'ออนไลน์', 'แจ้ง', 'เจ้าหน้าที่', 'คดี', 'คน', 'จำนวน', '์', 'เดินทาง', 'อ.', 'ดิจิทัล', 'ข่าว', 'ปลอม

In [38]:
digitalb_15 = lda_model(data=digitalb['article_tokenize'], num_topics=15)

LDA Model with 15 topics
Perplexity: -8.445469306790377
------------------------------------------------------------
Topic 1
['ปี', 'บาท', 'ล้าน', 'หุ้น', 'ราคา', 'บริษัท', 'สินทรัพย์', 'ดิจิทัล', 'คาด', 'การลงทุน', 'ไทย', 'ตลาด', 'ลดลง', 'ดอลลาร์', 'เพิ่มขึ้น', 'เดือน', 'นักลงทุน', 'สหรัฐ', 'เงิน', 'ลงทุน', 'ระดับ', 'ธุรกิจ', 'เศรษฐกิจ', 'สกุล', 'จีน', 'มูลค่า', 'กำไร', 'ซื้อ', 'รายได้', 'โท']
------------------------------------------------------------
Topic 2
['ดิจิทัล', 'คณะกรรมการ', 'ล้าน', 'บาท', 'ก.ล.ต.', 'วันที่', 'สินทรัพย์', 'สำนักงาน', 'บริษัท', 'หลักทรัพย์', 'ตลาดหลักทรัพย์', 'กำกับ', 'ดำเนินการ', 'โครงการ', 'ปี', 'เงิน', 'จำนวน', 'พิจารณา', 'กระทรวงการคลัง', 'กรณี', 'ไทย', 'ภาษี', 'งบประมาณ', 'เดือน', 'อี', 'กรมสรรพากร', 'ยื่น', 'ระบบ', 'ข้อมูล', 'เก็บภาษี']
------------------------------------------------------------
Topic 3
['ไทย', 'แบรนด์', '์', 'สินค้า', 'สร้าง', 'ธุรกิจ', 'ลูกค้า', 'บริษัท', 'อร', 'ปี', 'โลก', 'คน', 'ดิจิทัล', 'ทำ', 'สำหรับ', 'ตลาด', 'ประสบการณ์', 'ส'

In [41]:
digitalb_20 = lda_model(data=digitalb['article_tokenize'], num_topics=20)

LDA Model with 20 topics
Perplexity: -8.605848040229105
------------------------------------------------------------
Topic 1
['พล', 'อ.', 'รัฐมนตรี', 'รัฐบาล', 'รมว.', 'ประยุทธ์', 'เรื่อง', 'นายกรัฐมนตรี', 'เศรษฐกิจ', 'ประชาชน', 'ส.ส.', 'ว่าการ', 'ประเทศ', 'ดิจิทัล', 'พรรค', 'ไทย', 'สังคม', 'โอชา', 'นายกฯ', 'จันทร์', 'การประชุม', 'ประธาน', 'กระทรวง', 'ทำ', 'สรณ', 'กล่าวว่า', 'รองนายกรัฐมนตรี', 'ชัย', 'พงษ์', 'คน']
------------------------------------------------------------
Topic 2
['เกม', '์', 'อร', 'น', 'ดิจิทัล', 'ทีม', 'ค', 'เล่น', 'เม', 'เร', 'ร์', 'ประธานาธิบดี', 'คน', 'ริ', 'ล', 'ส', 'เด', 'บริษัท', 'โก', 'อ', 'สหรัฐ', 'เค', 'ปี', 'ระบบ', 'นิ', 'แด', 'เท', 'ช', 'นา', 'ออนไลน์']
------------------------------------------------------------
Topic 3
['ไทย', 'นักท่องเที่ยว', 'ประเทศ', 'การท่องเที่ยว', 'เดินทาง', 'ปี', 'ท่องเที่ยว', 'เที่ยว', 'จีน', 'เอเชีย', 'ททท.', 'กีฬา', 'วันที่', 'เมือง', 'คน', 'โรงแรม', 'ดิจิทัล', 'ล้าน', 'การแข่งขัน', 'โลก', 'สำหรับ', 'ภูเก็ต', 'ทีม', 'ต่างชาติ

In [43]:
digitalm_5 = lda_model(data=digitalb['article_tokenize'], num_topics=5)

LDA Model with 5 topics
Perplexity: -8.089737581025304
------------------------------------------------------------
Topic 1
['ดิจิทัล', 'เงิน', 'สกุล', 'ดอลลาร์', 'สหรัฐ', 'บิท', 'จีน', 'ราคา', 'ไทย', 'ป', 'ประเทศ', 'ริ', 'โควิด', 'ค', 'คอย', 'สินทรัพย์', 'ระบาด', 'น์', 'ระดับ', 'ปี', 'ตลาด', 'เหรียญ', 'เดือน', 'รายงาน', 'จุด', 'เศรษฐกิจ', 'โท', 'Bitkub', 'แพร่ระบาด', 'วันที่']
------------------------------------------------------------
Topic 2
['ไทย', 'ดิจิทัล', 'ธุรกิจ', 'เทคโนโลยี', 'ประเทศ', 'สร้าง', 'ปี', 'บริษัท', 'บริการ', 'ระบบ', 'พัฒนา', 'ข้อมูล', 'องค์กร', 'การพัฒนา', 'เศรษฐกิจ', 'โลก', 'อุตสาหกรรม', 'สำหรับ', 'ที่จะ', 'นวัตกรรม', 'อี', 'ทำ', 'กล่าวว่า', 'การทำงาน', 'โควิด', 'โครงการ', 'คน', 'รูปแบบ', 'การลงทุน', 'ล้าน']
------------------------------------------------------------
Topic 3
['บาท', 'ล้าน', 'ดิจิทัล', 'บริษัท', 'ปี', 'สลาก', 'สินทรัพย์', 'วันที่', 'คณะกรรมการ', 'กิจการ', 'กสทช.', 'ไทย', 'จำนวน', 'โทรคมนาคม', 'โครงการ', 'จำกัด', 'เดือน', 'สำนักงาน', 'การลงทุน', 

In [ ]:
digitalb_10 = lda_model(data=digitalb['article_tokenize'], num_topics=10)

LDA Model with 10 topics
Perplexity: -8.228768619937277
------------------------------------------------------------
Topic 1
['บาท', 'ปี', 'ล้าน', 'หุ้น', 'การลงทุน', 'บริษัท', 'ธุรกิจ', 'คาด', 'โครงการ', 'ไทย', 'ลงทุน', 'ดิจิทัล', 'รายได้', 'ราคา', 'เศรษฐกิจ', 'เพิ่มขึ้น', 'เติบโต', 'เดือน', 'กำไร', 'ลดลง', 'ตลาด', 'นักลงทุน', 'สินทรัพย์', 'ซื้อ', 'ไตรมาส', 'ประเทศ', 'ลด', 'ก.ล.ต.', 'มูลค่า', 'ขาย']
------------------------------------------------------------
Topic 2
['ข้อมูล', 'ดิจิทัล', 'รัฐมนตรี', 'วันที่', 'คณะกรรมการ', 'ประชาชน', 'หน่วยงาน', 'ดำเนินการ', 'สำนักงาน', 'ว่าการ', 'กระทรวง', 'ไทย', 'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม', 'อ.', 'ระบบ', 'เศรษฐกิจ', 'การประชุม', 'ชัย', 'เรื่อง', 'สังคม', 'ประเทศ', 'แห่งชาติ', 'พ.ศ.', 'รมว.', 'กฎหมาย', 'พล', 'ประธาน', 'รัฐบาล', 'พื้นที่', 'ศูนย์']
------------------------------------------------------------
Topic 3
['ดิจิทัล', 'ล้าน', 'สินทรัพย์', 'เงิน', 'สกุล', 'ดอลลาร์', 'ปี', 'บาท', 'ตลาด', 'มูลค่า', 'ราคา', 'บิท', 'ไทย', 'เหรียญ', 'ปร

In [ ]:
digitalb_15 = lda_model(data=digitalb['article_tokenize'], num_topics=15)

LDA Model with 15 topics
Perplexity: -8.458344313430754
------------------------------------------------------------
Topic 1
['ปี', 'บาท', 'ล้าน', 'หุ้น', 'ธุรกิจ', 'บริษัท', 'ไทย', 'การลงทุน', 'คาด', 'ราคา', 'สินทรัพย์', 'ตลาด', 'เพิ่มขึ้น', 'ลดลง', 'ลงทุน', 'ดิจิทัล', 'รายได้', 'เติบโต', 'เดือน', 'สินเชื่อ', 'ซื้อ', 'ธนาคาร', 'นักลงทุน', 'เศรษฐกิจ', 'ระดับ', 'กำไร', 'ก.ล.ต.', 'ลด', 'ขาย', 'ไตรมาส']
------------------------------------------------------------
Topic 2
['ไทย', 'ภาพยนตร์', 'ปี', 'ชม', 'สยาม', 'ผลงาน', 'งาน', 'ดิจิทัล', 'วันที่', 'รายการ', 'สร้างสรรค์', 'รูปแบบ', 'สื่อ', 'กิจกรรม', 'The', 'ประเทศ', 'นำเสนอ', 'สร้าง', 'โลก', 'ภาพ', 'สำหรับ', 'ศิลปิน', 'th', 'พิเศษ', 'คน', 'ศิลปะ', 'หนัง', 'ออนไลน์', 'ผู้ชม', 'เวลา']
------------------------------------------------------------
Topic 3
['กิจการ', 'กสทช.', 'โทรคมนาคม', 'คณะกรรมการ', 'ดิจิทัล', 'ดี', 'อี', 'สำนักงาน', 'วันที่', 'ข้อมูล', 'จี', 'คลื่น', 'แห่งชาติ', 'ไทย', 'ให้บริการ', 'เรื่อง', 'คลื่นความถี่', 'ประชาชน', 'หน่วย

In [ ]:
digitalb_20 = lda_model(data=digitalb['article_tokenize'], num_topics=20)

LDA Model with 20 topics
Perplexity: -8.60803632975048
------------------------------------------------------------
Topic 1
['อันดับ', 'เกม', '์', 'อร', 'ริ', 'ดิจิทัล', 'ค', 'ทีม', 'ส', 'คะแนน', 'เล่น', 'น', 'การแข่งขัน', 'เร', 'เด', 'ปี', 'ร์', 'สปอร์ต', 'ป', 'โก', 'อี', 'อ', 'ลิ', 'โท', 'ไทย', 'เก', 'ข้อมูล', 'ระบบ', 'เม', 'สเปอร์']
------------------------------------------------------------
Topic 2
['กิจการ', 'กสทช.', 'โทรคมนาคม', 'วันที่', 'คณะกรรมการ', 'คลื่น', 'แห่งชาติ', 'คลื่นความถี่', 'ประมูล', 'สำนักงาน', 'ดิจิทัล', 'ไทย', 'ดี', 'ดาวเทียม', 'อี', 'ดำเนินการ', 'โทรทัศน์', 'ใบอนุญาต', 'บาท', 'ประชาชน', 'การประมูล', 'ล้าน', 'บอร์ด', 'กระจายเสียง', 'บริษัท', 'เอส', 'เรื่อง', 'เมกะเฮิรตซ์', 'เดือน', 'เสนอ']
------------------------------------------------------------
Topic 3
['ข้อมูล', 'เรื่อง', 'ประชาชน', 'รัฐมนตรี', 'กฎหมาย', 'พล', 'อ.', 'รัฐบาล', 'ดิจิทัล', 'รมว.', 'ว่าการ', 'ดี', 'ส.ส.', 'สังคม', 'คน', 'ประยุทธ์', 'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม', 'วันที่', 'หน่วยงาน', 

In [42]:
#lda_vis(data=digitalb['article_tokenize'], num_topics=10)

### 2.3 NMF

In [ ]:
#data = econ_b['article_tokenize'].apply(lambda x:' '.join(x))
#cvec = CountVectorizer(token_pattern= "\b[A-zก-๙][A-z\.\-ก-๙]*\b")
#data = cvec.fit_transform(data)

In [80]:
def nmf(data=None, n_components=10, vectorizer=None):
    model = NMF(n_components=n_components)
    model.fit(data)
    features = model.transform(data)
    components = pd.DataFrame(model.components_,
                                columns=vectorizer.get_feature_names())
    # Topic & Words
    for i in range(n_components):
        topic = components.iloc[i]
        print(f'Top 20 words for topic {i+1} are:')
        print(topic.nlargest(20))
        print('\n')

In [81]:
nmf(data=digitalb_vec, n_components=10, vectorizer=vectorizerb)

Top 20 words for topic 1 are:
คน            2.574548
เรื่อง        2.016198
ตัวเอง        1.929180
ทำ            1.898609
ดู            1.861988
รู้           1.822228
ตัว           1.691304
ดี            1.663482
เวลา          1.574902
ทำงาน         1.572895
หา            1.550315
เหมือน        1.480947
บอ            1.469952
เกิดขึ้น      1.459087
ต้องการ       1.446389
งาน           1.404324
โลก           1.398809
เข้ามา        1.380072
เลือก         1.358568
เป็นเรื่อง    1.332212
Name: 0, dtype: float64


Top 20 words for topic 2 are:
วันที่          2.525559
ประชาชน         2.323858
คณะกรรมการ      2.300360
เรื่อง          2.092627
ดำเนินการ       2.045703
แห่งชาติ        1.982714
รัฐบาล          1.682365
รัฐมนตรี        1.672991
อ.              1.616741
ดี              1.605040
กล่าวว่า        1.602212
พล              1.565685
พิจารณา         1.539848
ว่าการ          1.539046
กรณี            1.477035
นายกรัฐมนตรี    1.469229
ดิจิทัล         1.468702
กฎหมาย          1.456492
ประธ

In [82]:
nmf(data=digitalm_vec, n_components=10, vectorizer=vectorizerm)

Top 20 words for topic 1 are:
เทคโนโลยี        3.043922
องค์กร           2.470993
ธุรกิจ           2.363364
บริษัท           2.324418
พัฒนา            2.231025
นวัตกรรม         2.192695
สร้าง            2.170502
การพัฒนา         2.170056
บริการ           2.169405
ระบบ             2.078219
ข้อมูล           2.069119
ที่จะ            2.007533
การทำงาน         1.947112
มีประสิทธิภาพ    1.890100
ก้าว             1.866855
ดิจิทัล          1.814457
ยุค              1.795194
รูปแบบ           1.793894
ให้บริการ        1.765269
ประเทศ           1.761184
Name: 0, dtype: float64


Top 20 words for topic 2 are:
วันที่                                 2.574451
รัฐมนตรี                               2.558903
ว่าการ                                 2.301150
อ.                                     2.121590
กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม    2.100760
นายกรัฐมนตรี                           1.983320
พล                                     1.940479
ประยุทธ์                               1.822880
จันทร์    

### 2.4 Top2Vec
- fast learn, learn and deep learn

In [ ]:
# min_count =
# ngram_vocab =
# embedding_model= (doc2vec)
# workers = cpu amount

In [9]:
digitalb_list = digitalb['article'].tolist()

In [10]:
# Create function that tokenize, remove stopwords and perform RegularExpression
def tokenize(txt):
  
    ''' 
    Description:
        A function that returns tokenized words with stop words and other regular expressions 
        that do not match English and Thai alphabets removed

    Parameters:
        txt = text string

    Return:
        word tokens as list
    '''

    stopwords = thai_stopwords()
    tokens = word_tokenize(txt, keep_whitespace = False)
    tokens = [token for token in tokens if token not in stopwords]
    tokens = [token for token in tokens if re.match(r'[A-Za-zก-\uDe5c]+', token)]
    return tokens

In [20]:
def top2vec(documents=None):
    top2vec = Top2Vec(documents=documents,
                      speed='fast-learn',
                      tokenizer=tokenize)
    topic_sizes, topic_nums =top2vec.get_topic_sizes()
    topic_words, word_scores, topic_nums = top2vec.get_topics(18)
    for words, scores, num in zip(topic_words, word_scores, topic_nums):
        print(num)
        print(f'Words: {words}')
    return topic_sizes, topic_nums,  topic_words, word_scores,

In [22]:
top2vec(digitalb['article'].tolist())

2022-11-14 16:29:13,623 - top2vec - INFO - Pre-processing documents for training
2022-11-14 16:31:15,912 - top2vec - INFO - Creating joint document/word embedding
2022-11-14 16:40:19,689 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-14 16:40:24,986 - top2vec - INFO - Finding dense areas of documents
2022-11-14 16:40:28,278 - top2vec - INFO - Finding topics


0
Words: ['องค์กร' 'ไอที' 'ทรานส์' 'ฟอ' 'โซลูชั่น' 'เคียว' 'ไอ' 'ค็อก' 'สฟอร์'
 'การทำงาน' 'ร์' 'คุกคาม' 'คลาวด์' 'ทิฟ' 'ซับซ้อน' 'ไฮบริด' 'นซัมแวร์'
 'การเปลี่ยนแปลง' 'เทคโนโลยี' 'ธิงส์(' 'ออราเคิล' 'เอ' 'ความยืดหยุ่น'
 'ปรับเปลี่ยน' 'ไซเบอร์' '์ท' 'ยืดหยุ่น' 'คอมพิวติ้ง' 'ไอบีเอ็ม' 'ซิ'
 'ชั่น' 'วิกฤติ' 'ตี้' 'แปซิฟิก' 'อย่างไรก็ดี' 'พนักงาน' 'ทั่วโลก'
 'ปัญญาประดิษฐ์' 'โอที' 'ธุรกิจ' 'กลยุทธ์' 'การประมวลผล' 'ความปลอดภัย'
 'มากขึ้น' 'ช่องโหว่' 'ภัย' 'เผชิญ' 'ดจ์' 'เลิร์น' 'โจมตี']
1
Words: ['เมกะเฮิรตซ์' 'กสทช.' 'คลื่นความถี่' 'กระจายเสียง' 'โทรทัศน์' 'เฮิรตซ์'
 'โทรคมนาคม' 'ประมูล' 'กิจการ' 'กสท' 'การประมูล' 'คลื่น' 'ตัณฑ' 'ฐากร'
 'กิกะ' 'ใบอนุญาต' 'ทีโอที' 'โครงข่าย' 'คณะกรรมการ' 'แห่งชาติ' 'ความถี่'
 'ทค.' 'ประชาพิจารณ์' 'MHz' 'สัมปทาน' 'บอร์ด' 'เรียกคืน' 'ย่าน' 'คม'
 'ที่ประชุม' 'บมจ.' 'bet' 'ไมโครโฟน' 'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม'
 'คณะรัฐมนตรี' 'เสนอ' 'เลขาธิการ' 'เยียวยา' 'วิทยุโทรทัศน์' 'การเยียวยา'
 'อสมท' 'มีมติ' 'กสท.' 'รัฐวิสาหกิจ' 'โทรศัพท์เคลื่อนที่' 'แบนด์'
 'ว

(array([508, 430, 409, 352, 351, 338, 287, 272, 266, 260, 242, 241, 235,
        230, 230, 229, 221, 213, 211, 202, 199, 183, 179, 172, 170, 169,
        165, 163, 161, 160, 158, 157, 151, 150, 149, 147, 143, 138, 137,
        135, 135, 135, 132, 131, 128, 127, 127, 127, 126, 125, 124, 122,
        121, 119, 118, 113, 110, 108, 108, 107, 106, 106, 104, 103,  96,
         93,  92,  92,  90,  89,  89,  89,  88,  88,  86,  86,  85,  85,
         83,  78,  78,  78,  77,  76,  72,  71,  71,  69,  66,  66,  66,
         65,  64,  64,  63,  63,  63,  62,  61,  60,  60,  60,  59,  58,
         57,  57,  57,  56,  55,  54,  53,  53,  53,  52,  50,  49,  49,
         48,  48,  47,  46,  45,  44,  44,  44,  42,  42,  41,  39,  38,
         37,  34,  33,  32,  32,  28,  28,  27,  26,  25,  23,  23,  22,
         19,  19,  17]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17]),
 array([['องค์กร', 'ไอที', 'ทรานส์', 'ฟอ', 'โซลูชั่น', 'เคียว', 'ไอ',
         'ค็

In [21]:
top2vec(digitalm['article'].tolist())

2022-11-14 16:22:42,872 - top2vec - INFO - Pre-processing documents for training
2022-11-14 16:23:54,630 - top2vec - INFO - Creating joint document/word embedding
2022-11-14 16:29:08,962 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-14 16:29:12,940 - top2vec - INFO - Finding dense areas of documents
2022-11-14 16:29:13,206 - top2vec - INFO - Finding topics


0
Words: ['คลื่นความถี่' 'กสทช.' 'เมกะเฮิรตซ์' 'การประมูล' 'กระจายเสียง' 'โทรทัศน์'
 'ประมูล' 'กิจการ' 'เฮิรตซ์' 'โอเปอเรเตอร์' 'โทรคมนาคม' 'ฐากร' 'ใบอนุญาต'
 'ซ์)' 'เรียกคืน' 'ตัณฑ' 'คลื่น' 'กิกะ' 'ย่าน' 'ทีวี' 'งวด' 'G' 'โครงข่าย'
 'ความถี่' 'โทรศัพท์เคลื่อนที่' 'MHz' 'จัดสรร' 'ภาคพื้นดิน' 'ค่าชดเชย'
 'อสมท' 'สปริง' 'ชำระ' 'ฐา' 'สัมปทาน' 'หลักเกณฑ์' 'ร์เลส' 'เงินชดเชย'
 'วิทยุโทรทัศน์' 'ช่อง' 'GHz' 'การออกอากาศ' 'บรอดแบนด์' 'ภาษีมูลค่าเพิ่ม'
 'แห่งชาติ' 'คอมมิวนิเคชั่น' 'การอนุญาต' 'เยียวยา' 'เชาวน์' 'เลขาธิการ'
 'มหาชน']
1
Words: ['กระทำความผิด' 'ดำเนินคดี' 'ความผิด' 'คำสั่งศาล' 'แจ้งความ' 'เข้าข่าย'
 'การกระทำ' 'พ.ต.อ.' 'พนักงานสอบสวน' 'บังคับการ' 'ปราบปราม' 'จับกุม' 'ท.)'
 'บิดเบือน' 'เจ้าหน้าที่ตำรวจ' 'สำนักงานตำรวจแห่งชาติ' 'การจับกุม'
 'ผิดกฎหมาย' 'พยานหลักฐาน' 'โพสต์' 'ปลอม' 'ปอ' 'ศาล' 'ตามกฎหมาย'
 'การปิดกั้น' 'มีความผิด' 'ประมวลกฎหมายอาญา' 'ผู้ต้องหา' 'ข้อความ' 'ระงับ'
 'เท็จ' 'พล.ต.ต.' 'ท.' 'ความร้ายแรง' 'ตำรวจ' 'ปิดกั้น' 'เอาผิด' 'ศาลอาญา'
 'ผู้เสียหาย' 'การสืบสวน' 'เฟซบุ๊

(array([372, 352, 307, 294, 282, 268, 252, 250, 243, 241, 230, 213, 213,
        198, 184, 167, 167, 161, 154, 141, 138, 136, 135, 135, 133, 133,
        132, 129, 127, 126, 125, 124, 117, 115, 113, 112, 110, 109, 105,
        103, 101,  92,  89,  88,  87,  84,  82,  80,  79,  78,  78,  78,
         77,  77,  75,  71,  71,  67,  66,  64,  62,  61,  59,  59,  59,
         58,  58,  58,  56,  53,  53,  51,  51,  50,  49,  47,  45,  45,
         44,  43,  41,  41,  40,  40,  39,  38,  37,  37,  36,  36,  34,
         33,  31,  29,  29,  29,  28,  23,  21]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17]),
 array([['คลื่นความถี่', 'กสทช.', 'เมกะเฮิรตซ์', 'การประมูล',
         'กระจายเสียง', 'โทรทัศน์', 'ประมูล', 'กิจการ', 'เฮิรตซ์',
         'โอเปอเรเตอร์', 'โทรคมนาคม', 'ฐากร', 'ใบอนุญาต', 'ซ์)',
         'เรียกคืน', 'ตัณฑ', 'คลื่น', 'กิกะ', 'ย่าน', 'ทีวี', 'งวด', 'G',
         'โครงข่าย', 'ความถี่', 'โทรศัพท์เคลื่อนที่', 'MHz', 'จัดสรร',
         '

In [11]:
top2vec = Top2Vec(documents=digitalb_list,
                  speed='fast-learn',
                  tokenizer=tokenize)

2022-11-14 15:46:09,283 - top2vec - INFO - Pre-processing documents for training
2022-11-14 15:47:58,060 - top2vec - INFO - Creating joint document/word embedding
2022-11-14 15:57:23,987 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-14 15:57:45,326 - top2vec - INFO - Finding dense areas of documents
2022-11-14 15:57:48,084 - top2vec - INFO - Finding topics


In [12]:
topic_sizes, topic_nums =top2vec.get_topic_sizes()
print(topic_sizes)
print(topic_nums)

[475 444 439 376 368 350 339 282 269 264 246 231 227 222 213 212 208 203
 180 179 177 177 176 173 172 172 171 170 170 169 167 166 164 164 161 161
 160 158 151 149 140 138 138 136 135 132 130 130 129 128 125 123 119 118
 117 117 114 113 111 110 109 104 102  99  98  96  96  95  94  93  91  89
  89  88  86  86  85  83  82  78  78  77  77  76  74  74  73  72  71  71
  70  66  65  64  62  62  60  59  58  57  57  57  56  54  54  54  52  51
  51  50  49  47  45  45  44  43  43  43  43  40  40  39  38  38  36  36
  36  34  34  33  33  32  29  29  27  25  25  25  24  24  23  22  20  17]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 1

In [13]:
topic_words, word_scores, topic_nums = top2vec.get_topics(18)
for words, scores, num in zip(topic_words, word_scores, topic_nums):
    print(num)
    print(f'Words: {words}')

0
Words: ['ประธานาธิบดี' 'โดนัลด์' 'ปูติน' 'โอบามา' 'กรุง' 'วอชิงตัน' 'สหรัฐ'
 'รัสเซีย' 'เกาหลีเหนือ' 'ซีเรีย' 'แถลงการณ์' 'ทรัมป์' 'เยอรมนี'
 'สำนักข่าว' 'ผู้อพยพ' 'คว่ำบาตร' 'อังกฤษ' 'กระทรวงการต่างประเทศ' 'ยุโรป'
 'อาเซียน' 'ผู้นำประเทศ' 'เรียกร้อง' 'อียู' 'ลิเบีย' 'การทูต' 'teamlab'
 'กองกำลัง' 'ตุรกี' 'ฝรั่งเศส' 'อาร์เจนตินา' 'การก่อการร้าย' 'ผู้ก่อเหตุ'
 'หวด' 'ปักกิ่ง' 'ผู้ต้องสงสัย' 'การตอบโต้' 'รอยเตอร์' 'สิทธิมนุษยชน'
 'สันติภาพ' 'เกาหลีใต้' 'ลงโทษ' 'ซาอุดีอาระเบีย' 'ก่อการร้าย' 'เบอร์ลิน'
 'ถ้อยแถลง' 'เยือน' 'สหพันธรัฐ' 'ข่าวกรอง' 'ทางทหาร' 'ยูเครน']
1
Words: ['กสทช.' 'เมกะเฮิรตซ์' 'คลื่นความถี่' 'กระจายเสียง' 'โทรทัศน์' 'ประมูล'
 'เฮิรตซ์' 'โทรคมนาคม' 'กิจการ' 'กสท' 'คลื่น' 'ฐากร' 'การประมูล' 'กิกะ'
 'ตัณฑ' 'ใบอนุญาต' 'โครงข่าย' 'ทีโอที' 'ทค.' 'แห่งชาติ' 'MHz' 'ความถี่'
 'บอร์ด' 'สัมปทาน' 'ย่าน' 'คณะกรรมการ' 'ประชาพิจารณ์' 'คม'
 'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม' 'เรียกคืน' 'กสท.' 'ที่ประชุม'
 'เสนอ' 'เยียวยา' 'คณะรัฐมนตรี' 'bet' 'ไมโครโฟน' 'วิทยุกระจายเสียง'
 'มีมติ' '

In [14]:
documents, document_scores, document_ids = top2vec.search_documents_by_topic(topic_num=0, 
                                                                             num_docs=10)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f'Document: {doc_id}, Score: {score}')
    print('-'*20)
    print(doc)
    print('-'*20)
    print()

Document: 3042, Score: 0.8407645225524902
--------------------
สำนักข่าวเกียวโดรายงานโดยอ้างแหล่งข่าวทางการทูตว่า ญี่ปุ่นปฏิเสธข้อเรียกร้องที่ต้องการให้นายชินโซ อาเบะ นายกรัฐมนตรีญี่ปุ่นเข้าร่วมวงประณามรัสเซียเช่นเดียวกับชาติตะวันตกต่อกรณีสงครามกลางเมืองในซีเรีย เนื่องจากเกรงว่า การดำเนินการดังกล่าวจะส่งผลกระทบด้านลบต่อการประชุมสุดยอดกับประธานาธิบดีวลาดิมีร์ ปูติน แห่งรัสเซีย แถลงการณ์ที่ได้มีการเผยแพร่เมื่อวันที่ 7 ธ.ค.ที่ผ่านมา ผู้นำอังกฤษ เยอรมนี ฝรั่งเศส อิตาลี แคนาดา และสหรัฐ ได้เรียกร้องให้มีการยุติการสู้รบในเมืองอเลปโปในทันที รวมทั้งประณามรัฐบาลซีเรียที่ก่อเหตุโจมตี ตลอดจนรัสเซียที่ให้การสนับสนุนซีเรียที่ขัดขวางการให้ความช่วยเหลือด้านมนุษยชน ผู้นำกลุ่มประเทศดังกล่าวยังได้เรียกร้องให้สหประชาชาติสอบสวนรายงานเรื่องการก่ออาชญากรรมสงคราม และรวบรวมหลักฐานเพื่อดำเนินการกับผู้ก่อเหตุ ในขณะเดียวกันก็ระบุว่า รัสเซียนั้นเข้ามาขวางการดำเนินงานของคณะมนตรีความมั่นคงในเมืองอเลปโป 'หัวเว่ย' ปักหมุดดิจิทัลไทย ชู 'คลาวด์' ปฏิรูปธุรกิจ สไปซ์ แอนด์ บาร์เลย์ ร้านใหม่สไตล์แกสโตร เลาจน์ ประกาศกรมอุตุฯ

### 2.5 BERTopic